In [ ]:
from qboard import Solver
import numpy as np

from config import PARAMS

In [ ]:
solver_name = 'dwave'
s = Solver(mode=f"remote:{solver_name}", params=PARAMS)

In [ ]:
Q = np.random.randn(5,5)

spins, energy = s.solve_qubo(Q, timeout=30)
spins, energy

[2021-10-23 17:56:21] Solver remote:dwave started
[2021-10-23 17:56:21] Server status - 0 active tasks
[2021-10-23 17:56:21] Start matrix upload
[2021-10-23 17:56:21] Upload completed, start solving
[2021-10-23 17:56:30] Found solution -5.321138 
[2021-10-23 17:56:30] Solver completed, solution type - feasible 


([1, 1, 1, 1, 1], -5.321137697166032)

$x_{ij} = 1$, если i-ый город посещён на j-ом шаге, соответсвующая переменная QUBO

In [ ]:
NUMBER_CITIES = 5

Q = np.zeros((NUMBER_CITIES**2, NUMBER_CITIES**2))
w = np.array([
    [0, 15, 17, 16, 16],
    [15, 0, 3, 17, 4],
    [17, 3, 0, 15, 3],
    [16, 17, 15, 0, 12],
    [16, 4, 3, 12, 0]
])

$\sum_{j=0}^{N - 1} \sum_{i,k=0}^{N - 1} w_{ik} x_{i,j} x_{k, j+1}$

In [ ]:
for j in range(NUMBER_CITIES):
    for i in range(NUMBER_CITIES):
        for k in range(NUMBER_CITIES):
            Q[NUMBER_CITIES*i + j, NUMBER_CITIES*k + ((j + 1) % NUMBER_CITIES)] += w[i, k]

$\sum_{j=0}^{N}\left(\sum_{i=0}^{N-1} x_{i j}-1\right)^{2}+\sum_{i=0}^{N-1}\left(\sum_{j=0}^{N} x_{i j}-1\right)^{2}$.
Заметим, что $(\sum_i x_{ij} - 1)^2 = - \sum_i x_{ij} + 2 \sum_{i > k} x_{ij} x_{kj} + const$ 

In [ ]:
fine1 = 20
fine2 = 20

for j in range(NUMBER_CITIES + 1):
    for i in range(NUMBER_CITIES):
        Q[NUMBER_CITIES*i + (j % NUMBER_CITIES), NUMBER_CITIES*i + (j % NUMBER_CITIES)] -= fine1
        for k in range(i + 1, NUMBER_CITIES):
            Q[NUMBER_CITIES*i + (j % NUMBER_CITIES), NUMBER_CITIES*k + (j % NUMBER_CITIES)] += 2*fine1
            
for i in range(NUMBER_CITIES):
    for j in range(NUMBER_CITIES + 1):
        Q[NUMBER_CITIES*i + (j % NUMBER_CITIES), NUMBER_CITIES*i + (j % NUMBER_CITIES)] -= fine2
        for k in range(j + 1, NUMBER_CITIES + 1):
            Q[NUMBER_CITIES*i + (j % NUMBER_CITIES), NUMBER_CITIES*i + (k % NUMBER_CITIES)] += 2*fine2

In [ ]:
spins, energy = s.solve_qubo(Q, timeout=30)
spins, energy

[2021-10-23 17:56:30] Found solution in cache -151.0


([1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0],
 -151.0)

In [ ]:
x = np.array(spins).reshape((NUMBER_CITIES, NUMBER_CITIES))
x

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.]])

In [ ]:
path = []
for j in range(NUMBER_CITIES + 1):
    for i in range(NUMBER_CITIES):
        if x[i, j % NUMBER_CITIES] == 1:
            path.append(str(i + 1))
print(f"\nPath: {'->'.join(path)}")


Path: 1->2->3->5->4->1


In [ ]:
number_to_cities = {
    0: 'Краснопресненская', 1: 'Кропоткинская', 2: 'Библиотека им. Ленина',
    3: 'Третьяковская', 4: 'Охотный ряд'
}

In [ ]:
with open(f'results/solution_{solver_name}.csv', 'w') as pf:
    csv_string = ','.join([number_to_cities[int(x) - 1] for x in path])
    pf.write(csv_string)

In [ ]:
with open(f'results/adjacency_matrix', 'w') as pf:
    pf.write(str(w))

In [ ]:
with open(f'results/qubo', 'w') as pf:
    pf.write(str(Q))